# Metrics

Here are common metrics that have been designed or adapted specifically for recommendation systems.

In [86]:
import pandas as pd
from IPython.display import HTML
from sklearn.datasets import make_blobs

## Example task

The following cell has generated a mapping that we will use as an example.  It is generated in the format `<user/item> <-> rating`. 

All the sources I checked describe how to estimate the performance of the models in the case of binary output, where there are "relevant" and "non-relevant". Despite the fact that in life often occur and I have met with tasks where the pair user/item put in correlation to non-binary values (ratings or even preferences expressed in spent money), for simplicity in the beginning let's consider the classical variant.

In [110]:
r_width = 10
r_height = 30
np.random.seed(10)

R, c = make_blobs(
    n_samples=r_height,
    n_features=r_width,
    centers=3,
    random_state=10
)
R = np.round((R-R.min())/(R.max()-R.min())).astype(int)

# genrating combinations of object/item to be empty
combination_counts = 20
nan_combinations = np.concatenate(
    [
        np.random.randint(0, R.shape[0], [combination_counts,1]),
        np.random.randint(0, R.shape[1], [combination_counts,1])
    ],
    axis=1
)

# replacing some combinations with emptys
R = R.astype("float32")
for comb in nan_combinations:
    R[tuple(comb)] = np.NaN

R_frame = pd.Series(
    R.ravel(),
    index = pd.MultiIndex.from_tuples(
            [
                (j,i) 
                for j in np.arange(R.shape[1]) 
                for i in np.arange(R.shape[0])
            ],
            names = ["object", "item"]
    ),
    name = "relevant"
).reset_index()

R_frame.sample(10)

,object,item,relevant
148,4,28,0.0
147,4,27,1.0
154,5,4,NaN
8,0,8,1.0
105,3,15,1.0
24,0,24,0.0
125,4,5,0.0
276,9,6,0.0
130,4,10,1.0
226,7,16,0.0


## precision@k